In [8]:
import cv2
import os
import numpy as np
import nbimporter
from Model_test import MobileNetv2
import json
import time
import tensorflow as tf

In [9]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_built_with_cuda())  # ต้อง True
print(tf.config.list_physical_devices('GPU'))  # ต้องเจอ GPU

2.19.0
False
[]


cmd

jupyter nbconvert --to script Model_test.ipynb

jupyter nbconvert --to script Evaluation.ipynb

In [10]:
def load_and_preprocess(src_dir, size=224):
    images = []
    labels = []
    class_names = sorted(os.listdir(src_dir))
    class_to_idx = {c: i for i, c in enumerate(class_names)}
    for class_name in class_names:
        class_path = os.path.join(src_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        for filename in os.listdir(class_path):
            file_path = os.path.join(class_path, filename)
            img = cv2.imread(file_path)
            if img is None:
                print(f"Warning: Can't read {file_path}")
                continue
            img_resized = cv2.resize(img, (size, size), interpolation=cv2.INTER_CUBIC)
            images.append(img_resized)
            labels.append(class_to_idx[class_name])
    images = np.array(images, dtype=np.float32) / 255.0  # Normalize
    labels = np.array(labels)
    return images, labels

In [11]:
if __name__ == "__main__":
    # โหลด train data
    X_train, y_train = load_and_preprocess('data/Training')

    # โหลด test data
    X_test, y_test = load_and_preprocess('data/Testing')

    model = MobileNetv2((224, 224, 3), k=8)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # เริ่มจับเวลา
    start_time = time.time()

    # train
    history = model.fit(X_train, y_train, batch_size=32, epochs=20, validation_data=(X_test, y_test), verbose=1)

    # จบการจับเวลา
    end_time = time.time()

    # คำนวณเวลารวม (วินาที → นาที)
    elapsed_time = end_time - start_time
    hours = elapsed_time // 3600
    minutes = (elapsed_time % 3600) // 60
    seconds = elapsed_time % 60

    print(f"Training time: {int(hours)} hr {int(minutes)} min {int(seconds)} sec")

    metrics = {
        'loss': history.history['loss'],
        'accuracy': history.history['accuracy'],
        'val_loss': history.history['val_loss'],
        'val_accuracy': history.history['val_accuracy'],
    }

    with open('train_history.json', 'w') as f:
        json.dump(metrics, f)

Epoch 1/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 265s 2s/step - accuracy: 0.2986 - loss: 1.9226 - val_accuracy: 0.1220 - val_loss: 2.3426
Epoch 2/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 223s 2s/step - accuracy: 0.4833 - loss: 1.4378 - val_accuracy: 0.1220 - val_loss: 2.9021
Epoch 3/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 220s 2s/step - accuracy: 0.5862 - loss: 1.1403 - val_accuracy: 0.1220 - val_loss: 3.1795
Epoch 4/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - accuracy: 0.6529 - loss: 0.9831 - val_accuracy: 0.1220 - val_loss: 3.8194
Epoch 5/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 220s 2s/step - accuracy: 0.7049 - loss: 0.8371 - val_accuracy: 0.1220 - val_loss: 3.0261
Epoch 6/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.7402 - loss: 0.7519 - val_accuracy: 0.2134 - val_loss: 3.2132
Epoch 7/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - accuracy: 0.7728 - loss: 0.6510 - val_accuracy: 0.4293 - val_loss: 2.8275
Epoch 8/20
100/100 ━━━━━━━━━━━━━━━━━━━━ 219s 2s/step - accuracy: 0.7962 - loss: 0.5898 - val_accu